In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [43]:
n_input = 784
n_classes = 10

# graph inputs
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [44]:
def conv2d(img, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img, w, strides=[1, 1, 1, 1], 
                                                  padding='SAME'),b))

def max_pool(img, k):
    return tf.nn.max_pool(img, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(_X, _weights, _biases, _dropout):
    # Reshape input picture
    _X = tf.reshape(_X, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(_X, _weights['wc1'], _biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = max_pool(conv1, k=2)
    # Apply Dropout
    conv1 = tf.nn.dropout(conv1, _dropout)

    # Convolution Layer
    conv2 = conv2d(conv1, _weights['wc2'], _biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = max_pool(conv2, k=2)
    # Apply Dropout
    conv2 = tf.nn.dropout(conv2, _dropout)

    # Fully connected layer
    # Reshape conv2 output to fit dense layer input
    dense1 = tf.reshape(conv2, [-1, _weights['wd1'].get_shape().as_list()[0]]) 
    # Relu activation
    dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, _weights['wd1']), _biases['bd1']))
    # Apply Dropout
    dense1 = tf.nn.dropout(dense1, _dropout) # Apply Dropout

    # Output, class prediction
    out = tf.add(tf.matmul(dense1, _weights['out']), _biases['out'])
    return out

In [45]:
# Store layers / weights

weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1' : tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2' : tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1' : tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out' : tf.Variable(tf.random_normal([1024, n_classes]))
}

bias = {
    'bc1' : tf.Variable(tf.random_normal([32])),
    'bc2' : tf.Variable(tf.random_normal([64])),
    'bd1' : tf.Variable(tf.random_normal([1024])),
    'out' : tf.Variable(tf.random_normal([n_classes]))
}

In [46]:
# create model 
pred = conv_net(x, weights, bias, keep_prob)

In [47]:
# define cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
# define opt
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)

In [48]:
# eval model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32)) 

In [49]:
init = tf.initialize_all_variables()

In [50]:
batch_size = 128
training_iters = 100000
dropout = .75

with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # fit
        sess.run(optimizer, feed_dict = {x: batch_xs, y: batch_ys, keep_prob: dropout})
        # print 
        if step % 20 == 0:
            # calculate accuracy
            acc = sess.run(accuracy, feed_dict = {x: batch_xs, y: batch_ys, keep_prob: 1.})
            # calculate batch loss
            loss = sess.run(cost, feed_dict = {x: batch_xs, y: batch_ys, keep_prob: 1.})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc)
        step += 1
        
    print "Optimization Finished!"
    # Calculate accuracy for 256 mnist test images
    print "Testing Accuracy:", sess.run(accuracy, feed_dict = {x: mnist.test.images[:256],
                                                               y: mnist.test.labels[:256],
                                                               keep_prob: 1.})
        

Iter 2560, Minibatch Loss= 17224.035156, Training Accuracy= 0.24219
Iter 5120, Minibatch Loss= 7003.300781, Training Accuracy= 0.59375
Iter 7680, Minibatch Loss= 5093.353516, Training Accuracy= 0.69531
Iter 10240, Minibatch Loss= 5800.869141, Training Accuracy= 0.68750
Iter 12800, Minibatch Loss= 3107.093750, Training Accuracy= 0.78906
Iter 15360, Minibatch Loss= 3744.595459, Training Accuracy= 0.75000
Iter 17920, Minibatch Loss= 3413.699951, Training Accuracy= 0.80469
Iter 20480, Minibatch Loss= 1306.788818, Training Accuracy= 0.89062
Iter 23040, Minibatch Loss= 3445.354248, Training Accuracy= 0.84375
Iter 25600, Minibatch Loss= 2773.633789, Training Accuracy= 0.89844
Iter 28160, Minibatch Loss= 1951.109497, Training Accuracy= 0.83594
Iter 30720, Minibatch Loss= 836.203552, Training Accuracy= 0.92969
Iter 33280, Minibatch Loss= 430.026550, Training Accuracy= 0.94531
Iter 35840, Minibatch Loss= 1913.869385, Training Accuracy= 0.85938
Iter 38400, Minibatch Loss= 1093.579346, Training Ac